In [ ]:
""" 
Programmers: Kellie Glasgow & Courtney Ward 
Languages: python, CSS 
Tools: pywebio, pandas, excel 
Runs on: Visual Studio Code with Jupyter 
Included Files: cookbook.xlsx 

Purpose: Create a cookbook application to add & view recipes in your own personalized recipe book. 

Pseudocode 
Main menu has two buttons to navigate to two features 
    Add recipe 
    View Recipes 
View Recipes 
    Access database to get recipe names & basic information 
    Display this information in a table 
    Dropdown menu to select recipe 
    Submit button to view selected recipe 
View Selected Recipe 
    Access database & display recipe information, ingredients, & steps 
Add Recipe 
    Have input fields to collect data about recipe from user 
    Validate & save data 
"""

In [1]:
###IMPORTS###
#to run a server, config allows us to change the general look, allows use of CSS
from pywebio import start_server, config
#allows input fields to remain on page after submission (Makes fields persistent)
from pywebio.pin import *
#allows use of input commands
from pywebio.input import *
#allows use of output commands
from pywebio.output import *
#Database management tool
import pandas as pd
#Allows pandas to write to excel
from pandas import ExcelWriter
#allows opening of excel files 
import openpyxl
#& csv files
import csv
#brings in real time - allows us to control length of load screens for messages to display
import time
#restructures strings to keep consistent formatting
import re

###DEFINING GLOBAL DATA FRAMES###
#Declaring global data frames
global ingredientsdf, recipedf, stepsdf, dfDisplay, filepath,ingredDisplaydf
#format floats as integers
pd.options.display.float_format = '{:,.0f}'.format
#Our db file path
filepath = "C:\\Users\\kyle\\Downloads\\cookbook.xlsx"

#Declaring & initializing global styles as string of CSS code to keep consistent formatting throughout
global headerStyle, promptStyle, instructionStyle, recipeHeadingStyle, ingredientListStyle
headerStyle = 'text-align: center; background-color: tan; padding-top: 40px; padding-bottom: 40px; color: brown; font-size: 50px; font-weight: bold'
promptStyle = 'text-align: center; color:black; font-size: x-large; font-weight: bold'
instructionStyle = 'margin-left: 100px; font-size = 35pt; font-weight: bold'
recipeHeadingStyle = 'margin-left: 400px; color: black; font-size: 40pt; font-weight: bold'
ingredientListStyle = 'margin-left: 100px; font-size 30pt'

#instantiating global data frames
ingredientsdf = pd.read_excel(filepath, sheet_name='Ingredients')
recipedf = pd.read_excel(filepath, sheet_name='Recipe')
stepsdf = pd.read_excel(filepath, sheet_name='Steps')
recipedf.index = recipedf.index + 1
ingredientsdf.index = ingredientsdf.index + 1
stepsdf.index = stepsdf.index + 1 
dfDisplay = recipedf.drop(columns=['RecipeID', 'numIngred','numInstruct'])
dfDisplay.rename(columns={'RecipeName': 'Recipe', 'Description': 'Description', 'tags':'Tag', 'foodCat':'Category', 'cuisine': 'Cuisine', 'prepTime': 'Prep Time', 'cookTime': 'Cook Time'}, inplace=True)
    
#opening user database so user login can be verified
with open("C:\\Users\\kyle\\Downloads\\userPassDB.csv", 'r',encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    cred_list = []
    for row in csv_reader:
        cred_list.append(row)

In [2]:
def print_header():
    ###Function to print running header & nav bar###   
    
    # Nav bar button to return to main menu 
    put_button(["Main Menu"], onclick=main_menu).style('text-align: left; margin-top: 20px')
    # App title header
    put_markdown('# The Busy Chef').style(headerStyle)

In [3]:
def main_menu():
    ###Layout for main menu###

    #Clear any prior output
    clear()
    with use_scope("main_menu", clear = True):
        #display main menu header at top of page
        put_markdown("# Welcome to the Busy Chef").style(headerStyle)
        #Prompt user to make selection
        put_text("What would you like to do?").style(promptStyle)
        #Define & layout buttons for options
        put_buttons(["View Recipes", 'Add Recipe'], onclick=menuButtons).style('text-align: center')


In [4]:
def menuButtons(btn_val):
    ###Tells main menu buttons what to do/what functions to call###

    #Launches View Recipes Page
    if btn_val == "View Recipes":
        view_recipes()
    #Launches Add recipe page
    elif btn_val == "Add Recipe":
        add_recipe()

In [5]:
def save_dfs():
    ###SAVE DATAFRAMES TO DATABASE - PLAN TO USE UPON CLOSING###
    global ingredientsdf, recipedf, stepsdf, dfDisplay, filepath,ingredDisplaydf
    #with pd.ExcelWriter(filepath) as writer:  
    #    recipedf.to_excel(filepath, sheet_name='Recipe')
    #    ingredientsdf.to_excel(filepath,sheet_name='Ingredients')
    #    stepsdf.to_excel(filepath,sheet_name='Steps')
    #ingredientsdf = pd.read_excel(filepath, sheet_name='Ingredients')
    #recipedf = pd.read_excel(filepath, sheet_name='Recipe')
    #stepsdf = pd.read_excel("C:\\Users\\Kyle\\Downloads\\cookbook.xlsx", sheet_name='Steps')
    with ExcelWriter(filepath, mode="a", engine="openpyxl",if_sheet_exists= 'replace') as writer:
        recipedf.to_excel(writer, sheet_name='Recipe',index=False)
        ingredientsdf.to_excel(writer,sheet_name='Ingredients',index=False)
        stepsdf.to_excel(writer,sheet_name='Steps',index=False)
    ingredientsdf = pd.read_excel(filepath, sheet_name='Ingredients')
    recipedf = pd.read_excel(filepath, sheet_name='Recipe')
    stepsdf = pd.read_excel(filepath, sheet_name='Steps')
    recipedf.index = range(1, len(recipedf)+1)
    ingredientsdf.index = range(1, len(ingredientsdf)+1)
    stepsdf.index = range(1, len(stepsdf)+1)
    dfDisplay = recipedf.drop(columns=['RecipeID', 'numIngred','numInstruct'])
    dfDisplay.rename(columns={'RecipeName': 'Recipe', 'Description': 'Description', 'tags':'Tag', 'foodCat':'Category', 'cuisine': 'Cuisine', 'prepTime': 'Prep Time', 'cookTime': 'Cook Time'}, inplace=True)

In [6]:
def view_recipes():
    ###Layout & data for View All Recipes screen###
    
    #Clear any prior output & display common header at top of page
    clear()
    print_header()
    
    global dfDisplay 
    dfDisplay = recipedf.drop(columns=['RecipeID', 'numIngred','numInstruct'])
    dfDisplay.rename(columns={'RecipeName': 'Recipe', 'Description': 'Description', 'tags':'Tag', 'foodCat':'Category', 'cuisine': 'Cuisine', 'prepTime': 'Prep Time', 'cookTime': 'Cook Time'}, inplace=True)
    
    put_markdown("## Recipes").style('text-align:center')
    put_html(dfDisplay.to_html(border=0))
    optionsList = []
    for i in range(1, len(dfDisplay) + 1):
        optionsList.append(i)
    response = select('Which recipe would you like to view?', options = optionsList)
    display_recipe(response)

In [7]:
def generate_instructions(response):
    ###Get & format data for recipe instructions###

    #Get instruction data
    stepsDisplaydf = stepsdf.loc[stepsdf['StepListID'] == response]
    stepsDisplaydf = stepsDisplaydf.drop(columns=['StepListID'])
    steps_str = stepsDisplaydf.to_string(header=False,
                            index=False,
                            index_names=False).split('\n')
    for i in range(len(steps_str)):
        steps_str[i] = ' '.join(steps_str[i].split())
    steps_str = '\n'.join(steps_str)
    return steps_str

In [8]:
def display_recipe(response):
    ###Get data for recipe and format & display it to page###

    #Clear any prior output & display common header at top of page
    clear()
    print_header()
    
    #Get data for recipe
    ingredDisplaydf = ingredientsdf.loc[ingredientsdf['IngredListID'] == response]
    ingredDisplaydf = ingredDisplaydf.drop(columns=['IngredListID'])
    ingredDisplaydf.index = range(1, len(ingredDisplaydf)+1)
    
    #format & display recipe
    text_recipe = "# {}".format(recipedf.iloc[response - 1]['RecipeName'])
    put_markdown(text_recipe).style('recipeHeadingStyle')
    #NOT USED: put_html(ingredDisplaydf.to_html(border=0))
    instructionsTxt = generate_instructions(response)
    put_markdown("### Ingredients").style(ingredientListStyle)
    put_column([put_html(ingredDisplaydf.to_html(border=0)).style(ingredientListStyle), None, put_text(instructionsTxt).style(instructionStyle)])
    #NOT USED: put_text(instructionsTxt)

In [9]:
def get_ingredients(num):
    ###Layout for user input screens to gather new recipe ingredients###
    
    #Clear any prior output & display common header at top of page
    clear()
    print_header()
    
    #String for recipe ingredient prompt with automatic incremented ingredient number
    inputStr = "Enter Ingredient Information for Ingredient {}".format(num + 1)
    #Displays input field & prompt for recipe ingredient, amount & measurement
    ingred_info = input_group(inputStr, [
        input('Ingredient Name', name = 'Name', type = TEXT),
        input('Ingredient Amount', name = 'Amount', type = TEXT),
        input('Ingredient Measurement(example: cup, teaspoon)', name ='Measurement', type = TEXT)
    ])
    #Update ingredient data for recipe 
    ingredHelp = {'IngredListID':len(recipedf)}
    ingred_info.update(ingredHelp)
    return ingred_info

In [10]:
def get_steps(num):
    ###Layout for user input screens to gather new recipe steps###

    #Clear any prior output & display common header at top of page
    clear()
    print_header()
    
    #String for recipe step prompt with automatic incremented step number
    inputStr = "Enter Step {}".format(num + 1)
    #Displays input field & prompt for recipe step 
    info = input_group(inputStr, [
        input('Instruction', name = 'Instruction', type = TEXT)
    ])
    step = num + 1
    step = str(step) + '.'
    #Update step data for recipe 
    stepsHelp = {'StepListID':len(recipedf),'StepNum': step}
    info.update(stepsHelp)
    return info

In [11]:
def add_recipe():
    ###Layout for user input screen to gather new recipe information###

    #Clear any prior output & display common header at top of page
    clear()
    print_header()
    
    #get information from user
    info = input_group("Please Enter your Recipe Information",[
        input('Enter Recipe Name', name = 'RecipeName', type = TEXT),
        input('Enter Recipe Description', name = 'Description', type = TEXT),
        radio('Select your Tags', options = ['Vegetarian', 'Heart Healthy', 'Protien Heavy', 'Party'], name = 'tags'),      
        input('Food Category (Example: Indian, American, Italian)', name = 'foodCat', type = TEXT),
        select('Cuisine', options = ['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer', 'Dessert'], name  = 'cuisine'),
        input('Enter the Prep Time for the Meal', name = 'prepTime', type = NUMBER),
        input('Enter the Cook Time for the Meal', name = 'cookTime', type = NUMBER),
        input('Enter the number of ingredients in recipe:', name = 'numIngred', type = NUMBER),
        input('Enter the number of steps in recipe:', name = 'numInstruct', type = NUMBER)
    ])
    #Getting dictionary of db attribute names for consistency & avoid db errors
    recipeHelp = {'RecipeID': len(recipedf) + 1}
    #Update dataframe with recipe input
    info.update(recipeHelp)
    recipedf.loc[len(recipedf) + 1] = info

    #Get each ingredient info
    for i in range(info['numIngred']):
        ingred = get_ingredients(i)
        ingredientsdf.loc[len(ingredientsdf) + 1] = ingred
    #Get each step info
    for i in range(info['numInstruct']):
        steps = get_steps(i)
        stepsdf.loc[len(stepsdf) + 1] = steps
    #Updating dataframe
    recipedf.index = recipedf.index + 1
    ingredientsdf.index = ingredientsdf.index + 1
    stepsdf.index = stepsdf.index + 1
    global dfDisplay 
    dfDisplay = recipedf.drop(columns=['RecipeID', 'numIngred','numInstruct'])
    dfDisplay.rename(columns={'RecipeName': 'Recipe', 'Description': 'Description', 'tags':'Tag', 'foodCat':'Category', 'cuisine': 'Cuisine', 'prepTime': 'Prep Time', 'cookTime': 'Cook Time'}, inplace=True)
    
    ###Save data###
    save_dfs()
    ###Go to main menu after submission###
    main_menu()


In [12]:
def login():
    ###LOGIN Screen - not currently implemented###
    credentials = input_group("loginInfo", [
        input("Username", name="username"),
        input("Password", name="password",
            type=PASSWORD,
            placeholder="Enter your password",
            help_text="Please contact IT Support for access.",
            required = True
            ),
        ])
    with put_loading(shape='border', color='dark').style('text-align:center'):
        for row in cred_list:
            if (row[0] == credentials['username']) & (row[1] == credentials['password']):

                add_recipe()
            else:
                put_text('Incorrect Username or Password. Please try again')
                time.sleep(5)
                clear()
                login()     

In [13]:
###PYWEBIO FUNCTION THAT OPENS UNUSED PORT ON NETWORK###
###Server starts with main_menu page###
start_server(main_menu, port = 8080, debug = True )

Running on all addresses.
Use http://10.91.82.71:8080/ to access the application


RuntimeError: This event loop is already running